# IBM Data Science Capstone Project
## Week 3


Install and import relevant libraries

In [1]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [75]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from bs4 import BeautifulSoup # for webscraping

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#map rendering library
import folium

print ('Library import complete')

Library import complete


## Part I Webscraping
### Requirements
As stated in assignment definition

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
4. If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

### Assumptions
1. https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' is an appropriate source of data. 
   This source is recommended on the week 3 discussion forums
   It is noted the format of the data does not exactly match the expectations set by the assignment.


**Scrape data from Wikipedia using BeautifulSoup, clean and create dataframe**

In [64]:
r=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=BeautifulSoup(r.text,'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    
    postalCode = row.p.text[:3]
    borough = (row.span.text).split('(')[0]
    
    if borough=='Not assigned':
        #as per requirement 2 ignore rows with borough not assigned
        pass
    else:
        
        neighbourhood = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        if neighbourhood=='Not assigned':
            # as per requirement 4 where neighbourhood is not assigned, but borough is, set it to be the same as borough
            # inspection of the data indicates this is superfluous as no such data exists. Code included for completeness of answer.
            neighbourhood==borough
        
        else:
        #as per requirement 1 set column headers to be PostalCode, Borough, and Neighborhood
            cell['PostalCode'] = postalCode
            cell['Borough'] = borough
            cell['Neighborhood'] = neighbourhood 
        
            table_contents.append(cell)

df=pd.DataFrame(table_contents)

#update Borough names as suggested in hints
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


**Remove duplicate postal codes, and set postal code as index**

In [40]:
#check to see how many duplicate postcodes are included
count=df.groupby(['PostalCode']).size()
duplicates= count[count>1]
duplicates.count()

0

In [58]:
#requirement 3 is not applicable as no duplicate post codes exist on link used - the data is already amalgamated as requested.
#the following code is included for completeness, but does not perform any transformation as no duplicate postcodes exist

# concatenate the string
df['Neighborhood'] = df.groupby(['PostalCode'])['Neighborhood'].transform(lambda x : ' '.join(x))
  
# drop duplicate data
df = df.drop_duplicates()   


In [65]:
df=df.set_index('PostalCode')
df.head()

,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Queen's Park,Ontario Provincial Government


**Show number of postal codes in final dataframe**

In [54]:
#requirement 6 - show size useing 'shape' in final cell
df.shape

(103, 2)

## Part II Add Longitude and Latitude

### Requirements
Get  get the geographical coordinates of the neighborhoods using the Geocoder package or using a provided csv file

CSV file: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv

In [145]:
dfll= pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
dfll.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
dfll.set_index('PostalCode', inplace=True)
dfll.head()


,Latitude,Longitude
PostalCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [73]:
toronto_data=df.merge(dfll, left_on='PostalCode', right_on='PostalCode')
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Part III Explore and cluster the neighbourhoods in Toronto

### Requirements
1. Use map visualisation
2. Conduct clustering
3. Repeat some of the analysis conducted for NY in the practice lab

**Create a map of Toronto with neighbourhoods marked**

First using geolocator to obtain co-ordinates of Toronto, then using folium to plot neighbourhoods on map

In [76]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.6534817, -79.3839347.


In [77]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [149]:
# @hidden_cell
CLIENT_ID = 'LHONXSF1T2PNUTZN2Q1S0RQNLFWUB1HJPLNCLUFKI52SKHLN' # your Foursquare ID
CLIENT_SECRET = 'CJPZ4NKRF1S2GKDHSWV0M1XVFSVHAGEAANR1HOZE4YN1RN21' # your Foursquare Secret


In [148]:
#credentials are in a hidden cell
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Get data of 100 venues near each neighbourhood (note this is based on 500m from single neighbourhood cooridinate rather than actual neighbourhood boundaries.)

In [79]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
       # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [80]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

print(toronto_venues.shape)
toronto_venues.head()

(2012, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [144]:
print('There are {} uniques categories of venue.'.format(len(toronto_venues['Venue Category'].unique())))

There are 255 uniques categories of venue.


In order to cluster similar neighbourhoods, pivot the the venue data to display venue types per neigbhourhood and normalise.

In [128]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [129]:

#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()



,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455


**Find the most common venue categories for each neighbourhood**

In [130]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [131]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Yoga Studio,Monument / Landmark,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Coffee Shop,Athletics & Sports,Pub,Sandwich Place,Playground,Mediterranean Restaurant,Men's Store,Metro Station
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pharmacy,Intersection,Bridal Shop,Shopping Mall,Sandwich Place,Fried Chicken Joint,Frozen Yogurt Shop,Sushi Restaurant
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Women's Store,Italian Restaurant,Comfort Food Restaurant,Pizza Place,Cupcake Shop,Pub,Café


**Cluster neighbourhoods using K-Means**
Use the normalised data of venues in each neighbourhood to conduct K-Means clustering. Merge this data back to the data showing the top 10 venue categories in each neighbourhood

In [132]:

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [133]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# view the merged data
toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,,,,,
M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Fast Food Restaurant,Park,Construction & Landscaping,Food & Drink Shop,Mediterranean Restaurant,Medical Center,Men's Store,Massage Studio,Moroccan Restaurant,Metro Station
M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Park,Bakery,Pub,Café,Restaurant,Performing Arts Venue,Event Space,Electronics Store,Sushi Restaurant
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Miscellaneous Shop,Boutique,Coffee Shop,Vietnamese Restaurant,Yoga Studio,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Diner,Burger Joint,Café,Sandwich Place,Bar,Bank
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,0.0,River,Park,Pool,Yoga Studio,Mexican Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Men's Store
M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1.0,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Indian Restaurant,Burrito Place,Gym,Gay Bar,Fast Food Restaurant,Mediterranean Restaurant
M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558,1.0,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Skate Park,Brewery



Clean the data - some postcodes have no venue data and so no cluster label, remove this data before plotting

In [134]:
# Check every row has a cluster
toronto_merged[toronto_merged['Cluster Labels'].isnull()]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,,,,,
M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M9N,York,Weston,43.706876,-79.518188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
#postcodes with no venue data have no cluster. Drop these, and check result
toronto_merged = toronto_merged.dropna(subset=['Cluster Labels'])
toronto_merged[toronto_merged['Cluster Labels'].isnull()]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,,,,,


**Visualise the neighbourhood clusters on a map**

In [136]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster= int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Almost all neighbourhoods are cluster 1, so lets look at the other clusters to see what kind of venues are in them.  Note clusters 3 and 4 are singletons!**

In [141]:
toronto_merged.loc[toronto_merged['Cluster Labels'] != 1].sort_values(by=['Cluster Labels'])

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,,,,,
M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Fast Food Restaurant,Park,Construction & Landscaping,Food & Drink Shop,Mediterranean Restaurant,Medical Center,Men's Store,Massage Studio,Moroccan Restaurant,Metro Station
M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,0.0,Jewelry Store,Park,Trail,Sushi Restaurant,Yoga Studio,Mexican Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station
M2P,North York,York Mills West,43.752758,-79.400049,0.0,Park,Convenience Store,Yoga Studio,Moroccan Restaurant,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577,0.0,Intersection,Playground,Park,Yoga Studio,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Metro Station
M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0.0,Photography Studio,Bus Line,Swim School,Park,Construction & Landscaping,Middle Eastern Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Metro Station
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,0.0,River,Park,Pool,Yoga Studio,Mexican Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Men's Store
M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,0.0,Park,Yoga Studio,Moroccan Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,0.0,Pizza Place,Park,Bus Line,Sandwich Place,Moroccan Restaurant,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
M6L,North York,"North Park, Maple Leaf Park, Upwood Park",43.713756,-79.490074,0.0,Bakery,Construction & Landscaping,Park,Basketball Court,Yoga Studio,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
